<a href="https://colab.research.google.com/github/Imperial-lord/sdn-controllers-load-balancing/blob/main/SDN_Load_Balancing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
####### Graph Building using data from Arnes.gml ########
import networkx as nx
import time
import random
import math
import numpy as np
import pandas as pd

from math import radians, cos, sin, asin, sqrt


k = int(input("Enter the number of controllers to allocate: "))
start = time.time()
graph = nx.read_gml('/content/drive/MyDrive/BTP Documents/Arnes.gml', label='id')

lat = nx.get_node_attributes(graph, "Latitude")
lon = nx.get_node_attributes(graph, "Longitude")

# Remove bad nodes from 'graph'
for node in list(graph.nodes):
    # removing nodes 
    # (a) with undefined latitude or longitude  
    # (b) that are isolated
    check_bad_node = lat.get(node) == None or lon.get(node) == None or nx.is_isolate(graph,node)
    if check_bad_node == True:
        graph.remove_node(node)


def dist_between_nodes(node_1, node_2):
    '''Compute distance between any 2 nodes of 'graph' using Haversine formula

    Args:
        node_1 (int): The first among the 2 graph nodes 
        node_2 (int): The second among the 2 graph nodes 

    Returns:
        dist: distanct between the two nodes
    '''
    lat_1 = radians(lat.get(node_1))
    lat_2 = radians(lat.get(node_2))
    lon_1 = radians(lon.get(node_1))
    lon_2 = radians(lon.get(node_2))

    # Applying Haversine formula
    dlon = lon_2 - lon_1
    dlat = lat_2 - lat_1
    a = sin(dlat/2)**2 + cos(lat_1)*cos(lat_2)*sin(dlon/2)**2
    c = 2*asin(sqrt(a))

    # Mean radius of Earth
    R = 6371.009
    dist = c*R

    return dist

# Assigning weights (based on distance) to the edges of the graph
for node in list(graph.nodes):
    for neighbor in list(graph.neighbors(node)):
        graph[node][neighbor]['weight'] = dist_between_nodes(node, neighbor)

# Removing edges with no weights assigned
for edge in list(graph.edges):
    if 'weight' not in graph.edges[edge] == False:
        graph.remove_edge(edge[0],edge[1])

print(nx.info(graph))
n = nx.number_of_nodes(graph)


# Store the graph as an adjacency list
adj_list = [];
for i in range(0,n):
    adj_list.append([])

for edge in graph.edges:
    i,j = edge
    adj_list[i].append(j)
    adj_list[j].append(i)



# Random sampling for controller index selection
controller = random.sample(range(0,n),k)

# List of lists represents index of switches in particular controller 
# e.g. [[s1,s2],[s3,s4],[s5]] controller distribution set
lol = []
for i in range(0,k):
    lol.append([])

# Controllers set
controller_max = list(controller)
print("\nControllers: \n{}\n".format(controller))

def BFS(queue, adj_list, listx):
    '''Performs a breadth first search in the graph
    '''
    while queue:
        p = queue.pop(0)
        u, w = p
        for v in adj_list[u]:
            if listx[v] == math.inf:
                listx[v] = w + 1
                queue.append((v, w + 1))
    return listx


index = 0
for j in range(0, k):
    listx = n*[math.inf]
    listx[controller[index]] = 0
    queue = []
    queue.append((controller[index], 0))
    listx = BFS(queue, adj_list, listx)
    lol[j] = listx
    index += 1


transpose_list = np.transpose(lol).tolist()
minumum_pos = []

for x in transpose_list:
    min_ele = n+5
    for i in range(0, len(x)):
        if x[i] < min_ele:
            min_ele = x[i]
    ts = []
    for i in range(0, len(x)):
        if(x[i] == min_ele):
            ts.append(i)
    minumum_pos.append(random.choice(ts))

final = []
controller_set = []
for i in range(0, k):
    controller_set.append([])

for i in range(0, n):
    final.append((i, controller[minumum_pos[i]]))
    controller_set[minumum_pos[i]].append(i)

print('Cluster set for each controller: \n{}\n'.format(controller_set))


# For Q-learning
controller_set_max = list(controller_set)

frame = pd.read_csv('/content/drive/MyDrive/BTP Documents/data.csv')
nodes = list(graph.nodes)
random.shuffle(nodes)
frame = frame.rename(columns=dict(zip(frame.columns, nodes)))

load_array = []
for i in frame.index:
    Load = dict(frame.loc[i])
    load_array.append(Load)

Enter the number of controllers to allocate: 20
Graph with 34 nodes and 46 edges

Controllers: 
[31, 3, 22, 32, 12, 14, 29, 2, 15, 8, 26, 25, 33, 16, 4, 13, 6, 27, 30, 17]

Cluster set for each controller: 
[[31], [0, 3, 7], [22], [32], [11, 12], [14, 18], [28, 29], [2, 21], [15], [5, 8, 9], [26], [24, 25], [33], [16], [4, 23], [13], [1, 6, 20], [27], [30], [10, 17, 19]]

